# 5DOF Robot with a shelf
In this notebook, we deploy our algorithm on a 5DOF robot interacting with a shelf. We demonstrate that combining Algorithm 2 in Appendix F of "Certified Polyhedral Decompositions of Collision-Free Configuration Space" with the certification algorithm given in Algorithm 1 can yield certified configuration-space regions that can reach a large amount of task space.

**Please copy a Mosek License to mosek.lic to run this** Failure to do so will result in the Scs solver being called which is too inaccurate for many of these problem.

Notebook runtime: about 30 minutes

In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
import os

In [3]:
#pydrake imports
import time

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver

# from pydrake.geometry.optimization import IrisInRationalConfigurationSpace

In [4]:
from pydrake.geometry.optimization import (CspaceFreePolytope,
                                               SeparatingPlaneOrder)
# CIrisSeparatingPlane

In [5]:
from pydrake.geometry.optimization import (CspaceFreePolytope, 
                                               SeparatingPlaneOrder)

In [6]:
from ur3e_demo import UrDiagram
import ur3e_demo
import visualization_utils as viz_utils


# Build and set up the visualization the plant
Click on the link at the bottom to view a visualization of the plant.

In [7]:
ur = UrDiagram(num_ur = 1, weld_wrist = True, add_shelf = True,
                 add_gripper = True)
diagram_context = ur.diagram.CreateDefaultContext()
diagram = ur.diagram.ForcedPublish(diagram_context)

plant_context = ur.plant.GetMyMutableContextFromRoot(
        diagram_context)
scene_graph_context = ur.scene_graph.GetMyMutableContextFromRoot(
    diagram_context)
inspector = ur.scene_graph.model_inspector()        
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(ur.plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(ur.plant.num_positions())

# The object we will use to perform our certification.
cspace_free_polytope = CspaceFreePolytope(ur.plant, ur.scene_graph,
                                          SeparatingPlaneOrder.kAffine, q_star)


INFO:drake:Meshcat listening for connections at http://localhost:7002


RuntimeError: Could not find Drake resource_path 'drake/manipulation/models/ur3e/ur3e_cylinder_weld_wrist.urdf' because Drake CMake install marker specified a resource root of '/usr/local/lib/python3.11/site-packages/pydrake/lib/../share' but that root did not contain the expected file '/usr/local/lib/python3.11/site-packages/pydrake/lib/../share/drake/manipulation/models/ur3e/ur3e_cylinder_weld_wrist.urdf'.

## Set up the sliders so we can move the plant around manually

You can use the sliders below to move the two degrees of freedom of the plant around. A green dot will appear in the TC-space visualization describing the current TC-space configuration.

In [8]:
sliders = []
for i in range(ur.plant.num_positions()):
    q_low = ur.plant.GetPositionLowerLimits()[i]
    q_high = ur.plant.GetPositionUpperLimits()[i]
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, description=f"q{i}"))

q = np.zeros(ur.plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    ur.plant.SetPositions(plant_context, q)
    ur.diagram.ForcedPublish(diagram_context)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


NameError: name 'ur' is not defined

In [ ]:
sliders = []
sliders.append(widgets.FloatSlider(min=-1.7, max=1.7, value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=-1.7, max=1.7, value=0, description='q1'))

q = [0.0, 0.0]
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_q(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

## Set up the solver for certifying the region. 

It is **very** important that you upload a MOSEK license into the file `mosek.lic` in order to run the rest of the notebook. If you do not, the solver `SCS` will be used which is much slower and likely will be too inaccurate to solve many of the problems.

In [ ]:
os.environ["MOSEKLM_LICENSE_FILE"] = "mosek.lic"
with open(os.environ["MOSEKLM_LICENSE_FILE"], 'r') as f:
    contents = f.read()
    mosek_file_not_empty = contents != ''
print(mosek_file_not_empty)

solver_id = MosekSolver.id() if MosekSolver().available() and mosek_file_not_empty else ScsSolver.id()


# set up the certifier and the options for different search techniques
solver_options = SolverOptions()
# set this to 1 if you would like to see the solver output in terminal.
solver_options.SetOption(CommonSolverOption.kPrintToConsole, 0)

# The options for when we search for new planes and positivity certificates given the polytopes
find_separation_certificate_given_polytope_options = CspaceFreePolytope.FindSeparationCertificateGivenPolytopeOptions()
find_separation_certificate_given_polytope_options.num_threads = -1
find_separation_certificate_given_polytope_options.verbose = False
find_separation_certificate_given_polytope_options.solver_options = solver_options
find_separation_certificate_given_polytope_options.ignore_redundant_C = False
find_separation_certificate_given_polytope_options.solver_id = solver_id

# The options for when we search for a new polytope given positivity certificates.
find_polytope_given_lagrangian_option = CspaceFreePolytope.FindPolytopeGivenLagrangianOptions()
find_polytope_given_lagrangian_option.solver_options = solver_options
find_polytope_given_lagrangian_option.ellipsoid_margin_cost = CspaceFreePolytope.EllipsoidMarginCost.kGeometricMean
find_polytope_given_lagrangian_option.search_s_bounds_lagrangians = True
find_polytope_given_lagrangian_option.ellipsoid_margin_epsilon = 1e-4
find_polytope_given_lagrangian_option.solver_id = solver_id



bilinear_alternation_options = CspaceFreePolytope.BilinearAlternationOptions()
bilinear_alternation_options.max_iter = 10
bilinear_alternation_options.convergence_tol = 1e-3
bilinear_alternation_options.find_polytope_options = find_polytope_given_lagrangian_option
bilinear_alternation_options.find_lagrangian_options = find_separation_certificate_given_polytope_options

binary_search_options = CspaceFreePolytope.BinarySearchOptions()
binary_search_options.find_lagrangian_options = find_separation_certificate_given_polytope_options
binary_search_options.scale_min = 1e-3
binary_search_options.scale_max = 1.0
binary_search_options.max_iter = 5


# Generate and Certify Regions

Around some nominal seed postures, we will grow certified regions using Algorithm 2 which we will later certify.

In [ ]:
# Some seedpoints
q_top_shelf = np.array([-0.63, -1.03, 0.17, -2.33, -0.73])
q_middle_shelf = np.array([-0.40, -0.28, -0.79,  0.94,  1.05])
q_bottom_shelf = np.array([-0.13, -1.18,  1.62, -0.43, 1.57])
seed_points_q = np.array([   
                              q_top_shelf,
                              q_middle_shelf,
                              q_bottom_shelf
                              ])

seed_points = np.array([Ratfk.ComputeSValue(q, q_star)for q in seed_points_q])
s_top_shelf = Ratfk.ComputeSValue(q_top_shelf, q_star)
s_middle_shelf = Ratfk.ComputeSValue(q_middle_shelf, q_star)
s_bottom_shelf = Ratfk.ComputeSValue(q_bottom_shelf, q_star)

In [ ]:
iris_regions = []
iris_ellipses = []

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.configuration_space_margin = 1e-3
iris_options.relative_termination_threshold = 0.001

for i, s in enumerate(seed_points):
    q = Ratfk.ComputeQValue(s, q_star)
    ur.plant.SetPositions(plant_context, q)
    r = IrisInConfigurationSpace(ur.plant, 
                                         plant_context,
                                         q_star, iris_options)
    iris_regions.append(r)
    iris_ellipses.append(r.MaximumVolumeInscribedEllipsoid())


### We now attempt to find the smallest uniform shrinkage of our region which we are capable of certifying

In [ ]:
binary_search_region_certificates_for_iris = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, initial_region) in enumerate(zip(seed_points, iris_regions)):
    print(f"starting seedpoint {i+1}/{len(iris_regions)}")
    time.sleep(0.2)    
    cert = cspace_free_polytope.BinarySearch(set(),
                                                    initial_region.A(),
                                                    initial_region.b(), 
                                                    np.array(s),
                                                    binary_search_options)
    binary_search_region_certificates_for_iris[tuple(s)] = cert.certified_polytope

In [ ]:
def walk_around_region(region, num_verts):
    walk = viz_utils.generate_walk_around_polytope(region, num_verts)
    # Set robot link color. Hightlight the body with the closest distance.
    num_samples = 100
    time_points = np.linspace(0, walk.end_time(), num_samples)
    for t in time_points:
        s = walk.value(t)
        q = Ratfk.ComputeQValue(s, q_star)
        pair, distance = ur3e_demo.closest_distance(ur, plant_context, q)
        ur.plant.SetPositions(plant_context, q)
        ur.diagram.ForcedPublish(diagram_context)
        time.sleep(0.1)

In [ ]:
region_top_shelf = binary_search_region_certificates_for_iris[tuple(s_top_shelf)]
walk_around_region(region_top_shelf, 20)

In [ ]:
region_middle_shelf = binary_search_region_certificates_for_iris[tuple(s_middle_shelf)]
walk_around_region(region_middle_shelf, 20)

In [ ]:
region_bottom_shelf = binary_search_region_certificates_for_iris[tuple(s_bottom_shelf)]
walk_around_region(region_bottom_shelf, 20)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=91412238-dcb6-419d-a849-ff4a6cfdefbd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>